In [81]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error

In [82]:
df_original = pd.read_csv('data/train.csv')
test_original = pd.read_csv('data/test.csv')

In [83]:
df_train, df_test = train_test_split(df_original, test_size=0.2, random_state=0)

In [84]:
x_train = df_train.select_dtypes(exclude='object').drop(columns=['SalePrice', 'Id'])
y_train = df_train['SalePrice']

In [85]:
x_test = df_test.select_dtypes(exclude='object').drop(columns=['SalePrice', 'Id'])
y_test = df_test['SalePrice']

In [132]:
XGBRegressor?

In [188]:
xgb_old = XGBRegressor(random_state=0, n_jobs=-1)
xgb_old.fit(x_train, y_train)

y_test_preds = xgb_old.predict(x_test)
mae = mean_absolute_error(y_test, y_test_preds)
print(mae)

17539.71776005993


In [176]:
xgb = XGBRegressor(random_state=0, n_jobs=-1)

In [177]:
gird_params = {'n_estimators': range(40, 60, 1)}

In [178]:
xgb_gird = GridSearchCV(xgb, gird_params, cv=10, n_jobs=-1)

In [181]:
%%time
xgb_gird.fit(x_train, y_train)
print(xgb_gird.best_score_)
print(xgb_gird.best_params_)

0.8622878326180738
{'n_estimators': 59}
CPU times: user 325 ms, sys: 38.7 ms, total: 364 ms
Wall time: 4.15 s


In [182]:
# 0.8639844244994478

In [183]:
xgb_tuned = xgb_gird.best_estimator_

In [186]:
%%time
scores = cross_val_score(xgb_tuned, x_train, y_train, scoring='neg_mean_absolute_error', cv=10)
print('Mean Absolute Error %2f' %(-1 * scores.mean()))

Mean Absolute Error 17140.910435
CPU times: user 540 ms, sys: 3.1 ms, total: 543 ms
Wall time: 542 ms


In [187]:
# xgb_tuned.fit(x_train, y_train)
y_test_preds = xgb_tuned.predict(x_test)
mae = mean_absolute_error(y_test, y_test_preds)
print(mae)

18030.069964683218


In [ ]:
# 17539.71776005993